# Oanda Error Debuging

In [36]:
import pandas as pd
import numpy as np
import tpqoa
from datetime import datetime, timedelta, timezone # Need to import timezone seperatly
import time

In [38]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [48]:
class ConTrader(tpqoa.tpqoa):
    def __init__(self, conf_file, instrument, bar_length, window, units):
        super().__init__(conf_file)
        self.instrument = instrument
        self.bar_length = pd.to_timedelta(bar_length)
        self.tick_data = pd.DataFrame()
        self.raw_data = None
        self.data = None 
        self.last_bar = None
        self.units = units
        self.position = 0
        self.profits = [] 
        
        #*****************add strategy-specific attributes here******************
        self.window = window
        #************************************************************************
    
    def get_most_recent(self, days = 5):
        
        print("\n--- Entering get_most_recent ---") # DEBUG
        loop_counter = 0 # DEBUG counter
        max_loops = 10 # DEBUG limit
        
        while True:
            loop_counter += 1 # DEBUG
            print(f"\nLoop Iteration: {loop_counter}") # DEBUG

            if loop_counter > max_loops: # DEBUG safety break
                print(f"DEBUG: Exiting loop after {max_loops} iterations to prevent infinite run.")
                # Decide what to do here - maybe raise an error or just return
                break
            
            time.sleep(2)
            now = datetime.now(timezone.utc).replace(tzinfo=None) # new (Python 3.12)
            now = now - timedelta(microseconds = now.microsecond)
            past = now - timedelta(days = days)

            print(f"Fetching history from {past} to {now}") # DEBUG

            try:
                df = self.get_history(instrument = self.instrument, start = past, end = now,
                                      granularity = "S5", price = "M", localize = False).c.dropna().to_frame()

            
                if df.empty:
                    print("Warning: get_history returned empty DataFrame. Waiting and retrying.") # DEBUG
                    continue # Skip rest of loop and try again
            
                df.rename(columns = {"c":self.instrument}, inplace = True)
                # DEBUG check data before resample
                print(f"  History data shape BEFORE resample: {df.shape}")
                if not df.empty:
                    print(f"  History data tail BEFORE resample:\n{df.tail(3)}")
                    
                df = df.resample(self.bar_length, label = "right").last().dropna().iloc[:-1]
                # DEBUG
                print(f"  Resampled data shape (after dropna, iloc): {df.shape}")
                if df.empty:
                    print("Warning: Resampled DataFrame is empty after dropna/iloc. Waiting and retrying.") # DEBUG could be the issue if df is empty
                    continue
                print(f"  Resampled data tail:\n{df.tail(3)}")
                      
                self.raw_data = df.copy()
                self.last_bar = self.raw_data.index[-1]
                # DEBUG
                current_check_time = datetime.now(timezone.utc).replace(tzinfo=None) 
                time_diff = current_check_time - self.last_bar
                print(f"  Current Check Time: {current_check_time}")
                print(f"  Last Bar Time:      {self.last_bar} (Type: {type(self.last_bar)})")
                print(f"  Bar Length:         {self.bar_length} (Type: {type(self.bar_length)})")
                print(f"  Time Difference:    {time_diff}")
                print(f"  Condition Check: (Time Diff < Bar Length?) : {time_diff < self.bar_length}")
                if time_diff < self.bar_length:
                    print("--- Exit condition met. Breaking loop. ---") # DEBUG
                    break # Exit the loop
                
                if pd.to_datetime(datetime.now(timezone.utc)) - self.last_bar < self.bar_length: # new (Python 3.12)
                    break

            except Exception as e:
                print(f"ERROR during get_history or processing: {e}") # DEBUG
                print("Waiting before retry...")
                # Depending on the error, you might want to break or handle differently
                time.sleep(5) # Wait longer if there was an error

        print("--- Exiting get_most_recent ---") # DEBUG
                
    def start_trading(self, days): # NEW
        
        self.get_most_recent(days)
        self.stream_data(self.instrument)
            
    def on_success(self, time, bid, ask):
        print(self.ticks, end = "\r", flush = True)
        
        # recent_tick = pd.to_datetime(time)
        try:
            # Attempt conversion assuming 'time' might be tz-aware
            recent_tick = pd.to_datetime(time).tz_convert('UTC').tz_localize(None)
        except TypeError:
            # If 'time' is already naive (or conversion fails), assume it's UTC
            recent_tick = pd.to_datetime(time) # Already naive
        
        # define stop
        if self.ticks >= 100:
            self.terminate_session(cause = "Scheduled Session End.")
            return
        
        # collect and store tick data
        df = pd.DataFrame({self.instrument:(ask + bid)/2}, 
                          index = [recent_tick])
        self.tick_data = pd.concat([self.tick_data, df]) 
        
        # if a time longer than the bar_lenght has elapsed between last full bar and the most recent tick
        if recent_tick - self.last_bar >= self.bar_length:
            self.resample_and_join()
            self.define_strategy()
            self.execute_trades()

        if recent_tick - self.last_bar >= self.bar_length:
            # DEBUG: Log when resampling is triggered
            print(f"\nDEBUG: Triggering resample. Recent Tick: {recent_tick}, Last Bar: {self.last_bar}, Diff: {recent_tick - self.last_bar}")
            self.resample_and_join()
            self.define_strategy()
            self.execute_trades()
            
    def resample_and_join(self):
        self.raw_data = pd.concat([self.raw_data, self.tick_data.resample(self.bar_length, 
                                                                          label="right").last().ffill().iloc[:-1]])
        self.tick_data = self.tick_data.iloc[-1:]
        self.last_bar = self.raw_data.index[-1]
        
    def define_strategy(self): # "strategy-specific"
        df = self.raw_data.copy()
        
        #******************** define your strategy here ************************
        df["returns"] = np.log(df[self.instrument] / df[self.instrument].shift())
        df["position"] = -np.sign(df.returns.rolling(self.window).mean())
        #***********************************************************************
        
        self.data = df.copy()
        
    def execute_trades(self):
        if self.data["position"].iloc[-1] == 1:
            if self.position == 0:
                order = self.create_order(self.instrument, self.units, suppress = True, ret = True)
                self.report_trade(order, "GOING LONG")  
            elif self.position == -1:
                order = self.create_order(self.instrument, self.units * 2, suppress = True, ret = True) 
                self.report_trade(order, "GOING LONG")  
            self.position = 1
        elif self.data["position"].iloc[-1] == -1: 
            if self.position == 0:
                order = self.create_order(self.instrument, -self.units, suppress = True, ret = True)
                self.report_trade(order, "GOING SHORT")  
            elif self.position == 1:
                order = self.create_order(self.instrument, -self.units * 2, suppress = True, ret = True)
                self.report_trade(order, "GOING SHORT")  
            self.position = -1
        elif self.data["position"].iloc[-1] == 0: 
            if self.position == -1:
                order = self.create_order(self.instrument, self.units, suppress = True, ret = True) 
                self.report_trade(order, "GOING NEUTRAL")  
            elif self.position == 1:
                order = self.create_order(self.instrument, -self.units, suppress = True, ret = True)
                self.report_trade(order, "GOING NEUTRAL")  
            self.position = 0
    
    def report_trade(self, order, going):  
        time = order["time"]
        units = order["units"]
        price = order["price"]
        pl = float(order["pl"])
        self.profits.append(pl)
        cumpl = sum(self.profits)
        print("\n" + 100* "-")
        print("{} | {}".format(time, going))
        print("{} | units = {} | price = {} | P&L = {} | Cum P&L = {}".format(time, units, price, pl, cumpl))
        print(100 * "-" + "\n")  
        
    def terminate_session(self, cause): # NEW
        self.stop_stream = True
        if self.position != 0:
            close_order = self.create_order(self.instrument, units = -self.position * self.units,
                                            suppress = True, ret = True) 
            self.report_trade(close_order, "GOING NEUTRAL")
            self.position = 0
        print(cause)

In [50]:
trader = ConTrader("oanda.cfg", "EUR_USD", "1min", window = 1, units = 100)

In [52]:
# This execution code needs to replaced.

# trader.get_most_recent()
# trader.stream_data(trader.instrument, stop = 100)
# if trader.position != 0:
#     close_order = trader.create_order(trader.instrument, units = -trader.position * trader.units,
#                                       suppress = True, ret = True) 
#     trader.report_trade(close_order, "GOING NEUTRAL")
#     trader.position = 0

In [54]:
trader.start_trading(days = 5)


--- Entering get_most_recent ---

Loop Iteration: 1
Fetching history from 2025-03-25 21:58:55 to 2025-03-30 21:58:55
  History data shape BEFORE resample: (42859, 1)
  History data tail BEFORE resample:
                           EUR_USD
time                              
2025-03-30 21:58:10+00:00  1.08164
2025-03-30 21:58:20+00:00  1.08165
2025-03-30 21:58:30+00:00  1.08166
  Resampled data shape (after dropna, iloc): (4284, 1)
  Resampled data tail:
                           EUR_USD
time                              
2025-03-30 21:56:00+00:00  1.08166
2025-03-30 21:57:00+00:00  1.08166
2025-03-30 21:58:00+00:00  1.08165
ERROR during get_history or processing: Cannot subtract tz-naive and tz-aware datetime-like objects.
Waiting before retry...

Loop Iteration: 2
Fetching history from 2025-03-25 21:59:07 to 2025-03-30 21:59:07
  History data shape BEFORE resample: (42870, 1)
  History data tail BEFORE resample:
                           EUR_USD
time                              
202

TypeError: Cannot subtract tz-naive and tz-aware datetime-like objects.

### Added Wrapper methodes for start and terminate Trading Sessions